In [ ]:
install.packages("XML", repos="http://cran.us.r-project.org")

In [ ]:
install.packages("nycflights13", repos="http://cran.us.r-project.org")

<html><b>Importing all the required libraries.</b></html>

In [ ]:
library(dplyr)
library(XML)
library(nycflights13)

<html><b>Creating table.</b></html>

In [ ]:
my_db <- src_sqlite("my_db.sqlite3", create = T)

<html><b>Getting data from the database.</b></html>

In [ ]:
flights_sqlite <- copy_to(my_db, flights, temporary = FALSE, indexes = list(
  c("year", "month", "day"), "carrier", "tailnum"))

In [ ]:
flights_sqlite <- tbl(nycflights13_sqlite(), "flights")

In [ ]:
flights_sqlite

<html><b>Getting data from the database.</b></html>

In [ ]:
tbl(my_db, sql("SELECT * FROM flights"))

<html><b>To fetch the data from the database depening on certain conditions or fetching only certaion columns.</b></html>

In [ ]:
select(flights_sqlite, year:day, dep_delay, arr_delay)
filter(flights_sqlite, dep_delay > 240)
arrange(flights_sqlite, year, month, day)
mutate(flights_sqlite, speed = air_time / distance)

<html><b>To pull down all the results use collect().</b></html>

In [ ]:
c1 <- filter(flights_sqlite, year == 2013, month == 1, day == 1)
c2 <- select(c1, year, month, day, carrier, dep_delay, air_time, distance)
c3 <- mutate(c2, speed = distance / air_time * 60)
c4 <- arrange(c3, year, month, day, carrier)
collect(c4)

<html><b> We can see the query dplyr has generated by looking at the query component of the object.</b></html>

In [ ]:
c4$query

<html><b>Transaltes an expresion to sql.</b></html>

In [ ]:
translate_sql(x)

In [ ]:
translate_sql("x")

In [ ]:
translate_sql(x == 1 && (y < 2 || z > 3))

<html><b>Group by used to group observations within a dataset. It breaks down a dataset into specified groups of rows.</b></html>

In [ ]:
by_tailnum <- group_by(flights_sqlite, tailnum)
delay <- summarise(by_tailnum,
  count = n(),
  dist = mean(distance),
  delay = mean(arr_delay)
)
delay <- filter(delay, count > 20, dist < 2000)
delay_local <- collect(delay)

<html><b>src_postgres() has five arguments: dbname, host, port, user and password. dplyr uses the RPostgreSQL package to connect to postgres databases.</b></html>

In [ ]:
flights_postgres <- tbl(src_postgres("nycflights13"), "flights")

<html><b>The following examples show how we can perform grouped filter and mutate operations with PostgreSQL. </b></html>

In [ ]:
daily <- group_by(flights_postgres, year, month, day)

# Find the most and least delayed flight each day
bestworst <- daily %>% 
  select(flight, arr_delay) %>% 
  filter(arr_delay == min(arr_delay) || arr_delay == max(arr_delay))
bestworst$query

<html><b>Rank each flight within a daily</b></html>

In [ ]:
ranked <- daily %>% 
  select(arr_delay) %>% 
  mutate(rank = rank(desc(arr_delay)))
ranked$query